# A Pipeline for Processing and Visualizing Madrid City GPS Traces
Author: Shubham Aryal

In [25]:
# Data processing libraries
import pandas as pd
import numpy as np

# XML parsing for GPX files
import xml.etree.ElementTree as ET

# Web scraping
import requests
from bs4 import BeautifulSoup
import re

# Database
import sqlite3

# Visualization
import folium
from branca.colormap import LinearColormap

# Utilities
import time

In [29]:
def setup_db():
    """
    Initialize SQLite database and create tables for GPS traces.
    Returns database connection.
    """
    conn = sqlite3.connect('osm_traces.db')
    cursor = conn.cursor()
    
    # Create traces table to store trace metadata
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS traces (
            id INTEGER PRIMARY KEY,
            tag TEXT,
            name TEXT
        )
    ''')
    
    # Create points table to store GPS coordinates
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS points (
            trace_id INTEGER,
            lat REAL,
            lon REAL,
            time TEXT,
            speed REAL,
            FOREIGN KEY(trace_id) REFERENCES traces(id)
        )
    ''')
    
    # Clean up old data for fresh start
    print("Cleaning up old database records...")
    cursor.execute("DELETE FROM points")
    cursor.execute("DELETE FROM traces")
    
    conn.commit()
    return conn

In [31]:
def parse_gpx_and_save(conn, trace_id, tag, name, gpx_xml):
    """
    Parse GPX XML content and save to database.
    
    Parameters:
        conn: Database connection
        trace_id: Unique trace identifier
        tag: Location tag (e.g., 'Madrid')
        name: Trace filename
        gpx_xml: Raw GPX XML content
    """
    root = ET.fromstring(gpx_xml)
    
    # GPX files use XML namespaces
    ns = {'gpx': 'http://www.topografix.com/GPX/1/1'}
    
    cursor = conn.cursor()
    
    # Insert trace metadata
    cursor.execute(
        "INSERT OR IGNORE INTO traces VALUES (?, ?, ?)",
        (trace_id, tag, name)
    )
    
    # Extract all GPS points from the GPX file
    points_to_insert = []
    
    for trkpt in root.findall('.//gpx:trkpt', ns):
        # Get latitude and longitude from attributes
        lat = float(trkpt.attrib.get('lat'))
        lon = float(trkpt.attrib.get('lon'))
        
        # Get timestamp
        time_elem = trkpt.find('gpx:time', ns)
        dt = time_elem.text if time_elem is not None else None
        
        # Get speed (sometimes in extensions)
        speed_elem = trkpt.find('.//gpx:speed', ns)
        speed = float(speed_elem.text) if speed_elem is not None else 0.0
        
        points_to_insert.append((trace_id, lat, lon, dt, speed))
    
    # Batch insert all points efficiently
    cursor.executemany(
        "INSERT INTO points VALUES (?, ?, ?, ?, ?)",
        points_to_insert
    )
    
    conn.commit()
    print(f"Saved {len(points_to_insert)} points for Trace {trace_id}")

In [33]:
def scrape_osm_traces(tag, target_count=5, min_points=2000):
    """
    Scrape GPS traces from OpenStreetMap for a given location tag.
    
    Parameters:
        tag: Location tag to search (e.g., 'Madrid')
        target_count: Number of traces to collect
        min_points: Minimum GPS points required per trace
    """
    conn = setup_db()
    base_url = "https://www.openstreetmap.org"
    search_url = f"{base_url}/traces/tag/{tag}"
    
    # Use browser-like headers to avoid blocking
    headers = {'User-Agent': 'Mozilla/5.0 (Educational Project)'}
    
    # Regex pattern to extract point count
    point_pattern = re.compile(r'([\d,]+)\s+points')
    
    found = 0
    page = 1
    
    print(f"--- Starting Scrape: {tag} (Min: {min_points} pts) ---")
    
    while found < target_count:
        print(f"Scanning Page {page}...")
        
        # Make HTTP request to OSM
        response = requests.get(search_url, headers=headers)
        if response.status_code != 200:
            break
        
        # Parse HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.select('table#trace_list tbody tr')
        
        if not rows:
            break
        
        # Process each trace in the table
        for row in rows:
            if found >= target_count:
                break
            
            row_text = row.get_text()
            
            if 'points' not in row_text:
                continue
            
            # Extract number of GPS points
            try:
                found_points = point_pattern.findall(row_text)
                point_count = int(found_points[0].replace(',', ''))
            except (ValueError, IndexError):
                continue
            
            # Check if trace meets minimum point threshold
            if point_count >= min_points:
                # Find trace link (skip image links)
                links = row.select('a[href*="/traces/"]')
                link_tag = next((l for l in links if l.get_text().strip()), None)
                
                if link_tag:
                    trace_name = link_tag.get_text().strip()
                    trace_id = link_tag['href'].split('/')[-1]
                    
                    print(f"Found: {trace_id} | {point_count} pts | {trace_name}")
                    
                    # Download GPX file
                    download_url = f"{base_url}/trace/{trace_id}/data"
                    gpx_resp = requests.get(download_url, headers=headers)
                    
                    if gpx_resp.status_code == 200:
                        parse_gpx_and_save(conn, trace_id, tag, trace_name, gpx_resp.content)
                        found += 1
                        time.sleep(1)  # Polite delay between requests
        
        # Check for next page
        next_page_link = soup.find('a', title="Older Traces")
        if next_page_link and next_page_link.get('href'):
            search_url = base_url + next_page_link['href']
            page += 1
        else:
            break
    
    conn.close()
    print(f"--- Finished: {found} traces processed ---")

In [35]:
# Collect 5 GPS traces from Madrid
scrape_osm_traces('Madrid', target_count=5, min_points=2000)

Cleaning up old database records...
--- Starting Scrape: Madrid (Min: 2000 pts) ---
Scanning Page 1...
Found: 11431647 | 3841 pts | Anillo_ciclista_Madrid.gpx
Saved 3841 points for Trace 11431647
Found: 6658591 | 18683 pts | entrenamiento_rebecos_pedriza_entrada_laberinto_11_03_23.gpx
Saved 18683 points for Trace 6658591
Found: 3729979 | 12537 pts | 20210706_090014.gpx
Saved 12537 points for Trace 3729979
Found: 3155397 | 2352 pts | san_lorenzo_de_el_escorial.gpx
Saved 2352 points for Trace 3155397
Found: 3035204 | 5912 pts | efowepvqqidgjyrw.gpx
Saved 5912 points for Trace 3035204
--- Finished: 5 traces processed ---


In [37]:
# Connect to database
conn = sqlite3.connect('osm_traces.db')

# Query all traces
df_traces = pd.read_sql_query("SELECT * FROM traces", conn)
print(f"Found {len(df_traces)} traces in the database:")
print(df_traces)

# Check point counts for each trace
print("\nPoint counts per trace:")
for index, row in df_traces.iterrows():
    trace_id = row['id']
    query = f"SELECT COUNT(*) as count FROM points WHERE trace_id = {trace_id}"
    count_df = pd.read_sql_query(query, conn)
    print(f"Trace {trace_id}: {count_df['count'][0]} points")

Found 5 traces in the database:
         id     tag                                               name
0   3035204  Madrid                               efowepvqqidgjyrw.gpx
1   3155397  Madrid                     san_lorenzo_de_el_escorial.gpx
2   3729979  Madrid                                20210706_090014.gpx
3   6658591  Madrid  entrenamiento_rebecos_pedriza_entrada_laberint...
4  11431647  Madrid                         Anillo_ciclista_Madrid.gpx

Point counts per trace:
Trace 3035204: 5912 points
Trace 3155397: 2352 points
Trace 3729979: 12537 points
Trace 6658591: 18683 points
Trace 11431647: 3841 points


In [38]:
# Get first point for map centering
sample_point = pd.read_sql_query("SELECT lat, lon FROM points LIMIT 1", conn)

# Initialize map centered on first GPS point
m = folium.Map(
    location=[sample_point.lat[0], sample_point.lon[0]],
    zoom_start=13,
    tiles="OpenStreetMap"
)

# Color palette for different traces
colors = ['blue', 'red', 'green', 'orange', 'purple']

# Plot each trace on the map
for index, row in df_traces.iterrows():
    trace_id = row['id']
    trace_name = row['name']
    
    # Query all points for this trace
    query = f"SELECT * FROM points WHERE trace_id = {trace_id}"
    df_points = pd.read_sql_query(query, conn)
    
    if df_points.empty:
        continue
    
    # Convert speed from m/s to km/h
    df_points['speed_kmph'] = df_points['speed'] * 3.6
    
    # Create list of coordinates
    coords = list(zip(df_points.lat, df_points.lon))
    
    # Add trace line to map
    folium.PolyLine(
        coords,
        color=colors[index % len(colors)],
        weight=4,
        opacity=0.7,
        popup=f"Trace: {trace_name} (ID: {trace_id})"
    ).add_to(m)

# Save map to HTML file
m.save("madrid_gps_traces.html")
conn.close()

# Display map
m